In [8]:
pip install requests openai google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [10]:
import io
import requests
import base64
import openai  # Ensure OpenAI API is installed

# Your API Keys
GOOGLE_API_KEY = "###"
OPENAI_API_KEY = "###"


In [12]:
def detect_landmark(image_path):
    """Detects landmarks in an image using Google Vision API."""
    
    # Read image file and encode it to base64
    with io.open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Google Vision API Endpoint
    vision_api_url = f"https://vision.googleapis.com/v1/images:annotate?key={GOOGLE_API_KEY}"

    # API Request Payload
    request_payload = {
        "requests": [{
            "image": {"content": encoded_image},
            "features": [{"type": "LANDMARK_DETECTION", "maxResults": 5}]
        }]
    }

    # Send API request
    response = requests.post(vision_api_url, json=request_payload)

    # Check response status
    if response.status_code == 200:
        response_data = response.json()
        
        # Print full API response for debugging
        print("Full API Response:", response_data)

        # Extract landmark details
        try:
            landmarks = response_data["responses"][0]["landmarkAnnotations"]
            if landmarks:
                return landmarks[0]["description"]  # Most relevant landmark
        except KeyError:
            return "No landmark detected."
    else:
        return f"API Error: {response.status_code}, {response.text}"


In [14]:
def get_landmark_description(landmark_name):
    """Fetches historical details about the landmark using OpenAI GPT API."""
    
    openai.api_key = OPENAI_API_KEY
    prompt = f"Give a short historical description of {landmark_name}."

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a historian providing details about landmarks."},
                  {"role": "user", "content": prompt}]
    )

    if "choices" in response and response["choices"]:
        return response["choices"][0]["message"]["content"]
    else:
        return "No historical data available !!"


In [18]:
# Example Usage
image_path = "Colosseo_2020.jpg"  # Change this to your image file
landmark_name = detect_landmark(image_path)

if "API Error" not in landmark_name and landmark_name != "No landmark detected.":
    print(f"Detected Landmark: {landmark_name}")
    historical_info = get_landmark_description(landmark_name)
    print(f"Historical Description: {historical_info}")
else:
    print("No landmark detected !!")


Full API Response: {'responses': [{'landmarkAnnotations': [{'mid': '/m/0d5qx', 'description': 'Colosseum', 'score': 0.968624, 'boundingPoly': {'vertices': [{}, {'x': 1200}, {'x': 1200, 'y': 841}, {'y': 841}]}, 'locations': [{'latLng': {'latitude': 41.890210200000006, 'longitude': 12.492230899999997}}]}]}]}
Detected Landmark: Colosseum
Historical Description: The Colosseum, officially known as the Flavian Amphitheatre, is a large amphitheater in the center of the city of Rome, Italy. Construction of this monumental structure began under the rule of the Emperor Vespasian in around 70-72 AD and was completed in 80 AD under his successor and heir, Titus. Further changes were made during the reign of Domitian (81-96). The Colosseum, capable of seating approximately 50,000 spectators, was used for gladiatorial contests, public spectacles, and dramas based on Classical mythology. The arena itself was 83 meters by 48 meters (272 ft by 157 ft). 

Despite suffering significant damage over the ce